In [18]:
obs = ("11","01","00","10","10","11","00")
start_metric = {'zero':0,'one': 0, 'two': 0,'three':0}
state_machine = {
    #current state, current state로 들어오는 branches, branch information
    'zero': {'b1': {'out_b':"11",'prev_st': 'one','input_b':0},
             'b2': {'out_b':"00",'prev_st': 'zero','input_b':0}},
    'one': {'b1': {'out_b': "10", 'prev_st': 'three', 'input_b': 0},
             'b2': {'out_b': "01", 'prev_st': 'two', 'input_b': 0}},
    'two': {'b1': {'out_b': "00", 'prev_st': 'one', 'input_b': 1},
             'b2': {'out_b': "11", 'prev_st': 'zero', 'input_b': 1}},
    'three': {'b1': {'out_b': "01", 'prev_st': 'three', 'input_b': 1},
             'b2': {'out_b': "10", 'prev_st': 'two', 'input_b': 1}},
 
}

In [19]:
V = [{}]
for i in range(2):
    V.append({})
    for st in state_machine:# 4번 반복, zero, one, two, three
        # Calculating the probability of both initial possibilities for the first observation
        V[i][st] = {"metric": start_metric[st]}

In [21]:
def bits_diff_num(num_1,num_2): # Hamming Distance
    count=0;
    for i in range(0,len(num_1),1):
        if num_1[i]!=num_2[i]:
            count=count+1
    return count

In [22]:
t=2
V[t]['zero'] = {"metric" : bits_diff_num(state_machine['zero']['b2']['out_b']+state_machine['zero']['b2']['out_b'], obs[t - 2]+obs[t-1]), "branch1" : 'b2',"branch2" : 'b2'}
V[t]['one'] = {"metric" : bits_diff_num(state_machine['two']['b2']['out_b']+state_machine['one']['b2']['out_b'], obs[t - 2]+obs[t-1]),"branch1" : 'b2',"branch2" : 'b2'}
V[t]['two'] = {"metric" : bits_diff_num(state_machine['zero']['b2']['out_b']+state_machine['two']['b2']['out_b'], obs[t - 2]+obs[t-1]),"branch1" : 'b2',"branch2" : 'b2'}
V[t]['three'] = {"metric" : bits_diff_num(state_machine['two']['b2']['out_b']+state_machine['three']['b2']['out_b'], obs[t - 2]+obs[t-1]),"branch1" : 'b2',"branch2" : 'b2'}

In [23]:
for t in range(3, len(obs)+1): # 3 ~ 7
    V.append({}) # V에 추가
    for st in state_machine: # 4번 반복, zero, one, two, three
        #Check for smallest bit difference from possible previous paths, adding with previous metric
        prev_st = state_machine[st]['b1']['prev_st']
        first_b_metric = V[t-1][prev_st]["metric"] + bits_diff_num(state_machine[st]['b1']['out_b'], obs[t - 1]) # V[t-1][prev_st]["metric"] = 이전까지의 Hamming Distance 의 값, branch1 로 들어온 경우 
        prev_st = state_machine[st]['b2']['prev_st']
        second_b_metric = V[t-1][prev_st]["metric"] + bits_diff_num(state_machine[st]['b2']['out_b'], obs[t - 1])# branch2 로 들어온 경우
        #print(state_machine[st]['b1']['out_b'],obs[t - 1],t)
        if first_b_metric > second_b_metric:
            V[t][st] = {"metric" : second_b_metric,"branch":'b2'}
        else:
            V[t][st] = {"metric": first_b_metric, "branch": 'b1'}

In [33]:
Decode_code = []
smaller = min(V[len(obs)][st]["metric"] for st in state_machine) # Minimum Hamming Distance 값 반환
#traceback the path on smaller metric on last trellis column
for st in state_machine:
    if V[len(obs)][st]["metric"] == smaller:
        for t in range(len(obs),1,-1):
            if t==2:
                branch = V[t][st]["branch2"]
                Decode_code.insert(0, state_machine[st][branch]['input_b'])
                st = state_machine[st][branch]['prev_st']
                branch = V[t][st]["branch1"]
                Decode_code.insert(0, state_machine[st][branch]['input_b'])
            else:
                branch = V[t][st]["branch"]
                Decode_code.insert(0, state_machine[st][branch]['input_b'])
                st = state_machine[st][branch]['prev_st']
divided_lists = [Decode_code[i:i+7] for i in range(0, len(Decode_code), 7)] # Minimum Hamming Distance 값을 가지는 state가 여러개일 경우
for i, sublist in enumerate(divided_lists): # list를 나눈다
    print(f"Decoded_code : {sublist}, Message_code : {sublist[:5]}")

Decoded_code : [1, 0, 1, 1, 0, 0, 0], Message_code : [1, 0, 1, 1, 0]
